# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [44]:
!pip install pycryptodome
import Crypto.Util.number as cu
import Crypto.Random as rand
import hashlib as hl
import math


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimportowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [2]:
nlen = 2048

In [3]:
def lcm(a, b):
    return abs(a * b) // math.gcd(a, b)

In [4]:
def generate_keys():
    p2_256 = pow(2, 256)
    p2_nlen_1_2 = pow(2, (nlen - 1)//2)
    p2_nlen_2_100 = pow(2, nlen/2 - 100)
    p2_nlen_2 = pow(2, nlen//2)
    
    while True:
        p, q = cu.getPrime(nlen//2), cu.getPrime(nlen//2)
        if p > p2_nlen_1_2 and q > p2_nlen_1_2 and abs(p - q) > p2_nlen_2_100:
            break
    
    n = p * q
    f = (p - 1) * (q - 1)
    
    while True:
        e = cu.getRandomRange(65537, min(p2_256, f) - 1)
        if math.gcd(f, e) == 1 and lcm(p - 1, q - 1) >= e * p2_nlen_2:
            break

    d = cu.inverse(e, f)

    return n, e, d

In [5]:
generate_keys()

(14701818912750881522615272042200914939524841691743643074254753934284569311482691662097556636725676944173494197996337043243456616788738550166809457328742873350156446251424499279406175169574541683758432332673882597995934011839979117211340350310559941931092258891265014011628476034575072119662229644220406387800643811610136709270997142367960601604502749525772304473255752048018067730946022819173722307064478206634068239466255292687322938440943759749897976225717260503090754211010444076811997336510299986122973978323097554738490727146553372986906645059854890281206938123205409568735861867911334434303267173685636183407719,
 69763697894511889391392176901045726854863803071572604317459736350590907928203,
 11369404855169402409244592390465792267183619258136995521095531087693736639256066461698192347612539034207284041208013325509177242848308267295629337253965321518368744847820525342075187139174312337245089951282658974080739481421755729779019743738158933876776283457676184359848783099402240785934005566554

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [6]:
def as_int(b):
    return int.from_bytes(b, byteorder='big')

def as_bytes(i, l=None):
    if l is None:
        l = (math.ceil(math.log2(i)) + 1)//8
    return i.to_bytes(l, byteorder='big')

In [7]:
def rsa_encode(M, e, n):
    M_i = as_int(M)
    return pow(M_i, e, n)
    
def rsa_decode(C, d, n):
    M_i = pow(C, d, n)
    return as_bytes(M_i)

In [8]:
M = b"Ala ma kota"

In [9]:
n, e, d = generate_keys()

In [10]:
C = rsa_encode(M, e, n)

In [11]:
D = rsa_decode(C, d, n)

In [12]:
print(D.decode("utf-8"))

Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [13]:
def sha256(m):
    s = hl.sha256()
    s.update(m)
    return s.digest()

In [14]:
def sign(M, d, n):
    h = as_int(sha256(M))
    sig = pow(h, d, n)
    return sig

def verify(M, sig, e, n):
    h = as_int(sha256(M))
    ver = pow(sig, e, n)
    return ver == h

In [15]:
n, e, d = generate_keys()

In [16]:
sig = sign(M, d, n)

In [17]:
verify(M, sig, e, n)

True

In [18]:
sig = sig - 1

In [19]:
verify(M, sig, e, n)

False

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [20]:
H = sha256

In [21]:
hlen = 32

In [22]:
def mgf1(M, l):
    output = bytes()
    for i in range(math.ceil(l/hlen)):
        tmp = H(M + i.to_bytes(4, byteorder='big'))
        output = output + tmp
    return output[:l]

In [23]:
mgf1(M, 50)

b'O\x82y,\xd5\x98\x18\xc5B7\xc4YT\xe6\xfa\x82\xdd\x08\xc8\x8d\x9b?\xb2\xe2X3\x1c\tf%d\x18\xd6,\xad\x91\x8eH\x9e\xa6\x1b\xf7m\xd4\xfcu\xf5L\x0f&'

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpakuj tablicę bajtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [24]:
def bytes_xor(b1, b2):
    return as_bytes(as_int(b1) ^ as_int(b2), len(b1))

In [25]:
def oaep_encode(M, k):
    mlen = len(M)
    if mlen > k - 2*hlen - 2:
        return
    l_hash = H(b"")
    
    PS = bytearray(k - mlen - 2*hlen - 2)
    DB = l_hash + PS + b"\x01" + M
    seed = rand.get_random_bytes(hlen)
    
    db_mask = mgf1(seed, k - hlen - 1)
    masked_db = bytes_xor(DB, db_mask)
    
    seed_mask = mgf1(masked_db, hlen)
    masked_seed = bytes_xor(seed, seed_mask)
    
    EM = b"\x00" + masked_seed + masked_db
    
    return EM

def oaep_decode(EM, k):
    if EM[0] != 0x00:
        return
    masked_seed, masked_db = EM[1:hlen + 1], EM[hlen + 1:]
    l_hash = H(b"")
    
    seed_mask = mgf1(masked_db, hlen)
    seed = bytes_xor(masked_seed, seed_mask)
    
    db_mask = mgf1(seed, k - hlen - 1)
    DB = bytes_xor(masked_db, db_mask)
    
    l_hash_p = DB[:hlen]
    
    i = hlen
    while True:
        if DB[i] != 0x00:
            break
        i += 1
    if DB[i] != 0x01:
        return
    if l_hash_p != l_hash:
        return
    
    return DB[i + 1:]

In [26]:
M = b"Ala ma kota"

In [27]:
EM = oaep_encode(M, 1000)
print(oaep_decode(EM, 1000))

b'Ala ma kota'


In [28]:
def rsa_encode_oaep(M, e, n, k):
    EM = oaep_encode(M, k)
    EM_i = as_int(EM)
    return pow(EM_i, e, n)
    
def rsa_decode_oaep(C, d, n, k):
    EM_i = pow(C, d, n)
    EM = as_bytes(EM_i, k)
    return oaep_decode(EM, k)

In [29]:
n, e, d = generate_keys()

In [30]:
C = rsa_encode_oaep(M, e, n, 250)

In [31]:
D = rsa_decode_oaep(C, d, n, 250)

In [32]:
print(D.decode("utf-8"))

Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest podpis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [33]:
def bytes_and(b1, b2):
    return as_bytes(as_int(b1) & as_int(b2), len(b1))

In [34]:
slen = hlen
em_bits = nlen - 1
emlen = nlen//8

In [35]:
def emsa_encode(M):
    m_hash = H(M)
    if emlen < hlen + slen + 2:
        return
    salt = rand.get_random_bytes(slen)
    
    M_p = b"\x00"*8 + m_hash + salt
    m_hash_p = H(M_p)
    
    PS = bytearray(emlen - slen - hlen - 2)
    DB = PS + b"\x01" + salt
    
    db_mask = mgf1(m_hash_p, emlen - hlen - 1)
    masked_db = bytes_xor(DB, db_mask)
    
    EM = b"\x00"*(8*emlen - em_bits) + masked_db[8*emlen - em_bits:] + m_hash_p + b"\xbc"
    
    return EM

def emsa_verify(M, EM):
    m_hash = H(M)
    if emlen < hlen + slen + 2:
        return False
    if EM[-1] != 0xbc:
        return False
    
    masked_db, m_hash_p = EM[:emlen - hlen - 1], EM[emlen - hlen - 1:-1]
    if masked_db[:8*emlen - em_bits] != b"\x00"*(8*emlen - em_bits):
        return False
    
    db_mask = mgf1(m_hash_p, emlen - hlen - 1)
    DB = bytes_xor(masked_db, db_mask)
    DB = b"\x00"*(8*emlen - em_bits) + DB[8*emlen - em_bits:]
    if DB[:emlen - hlen - slen - 2] != b"\x00"*(emlen - hlen - slen - 2) or DB[emlen - hlen - slen - 2] != 0x01: # Był błąd w opisie, powinno być -2 w 2 warunku
        return False
    
    salt = DB[-slen:]
    
    M_p = b"\x00"*8 + m_hash + salt
    m_hash_pp = H(M_p)
    
    return m_hash_p == m_hash_pp

In [36]:
M = b"Ala ma podpisy"

In [37]:
def sign_emsa(M, d, n):
    EM = emsa_encode(M)
    sig = pow(as_int(EM), d, n)
    return sig

def verify_emsa(M, sig, e, n):
    EM = as_bytes(pow(sig, e, n), emlen)
    return emsa_verify(M, EM)

In [38]:
n, e, d = generate_keys()

In [39]:
sig = sign_emsa(M, d, n)
# sig -= 1
print(verify_emsa(M, sig, e, n))

True


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [40]:
def generate_sym_key(n):
    r = rand.get_random_bytes(256)
    key = H(r)
    return key

def encode_sym_key(key, e, n):
    ciphered_key = rsa_encode_oaep(key, e, n, 256)
    return ciphered_key

def decode_sym_key(ciphered_key, d, n):
    key = rsa_decode_oaep(ciphered_key, d, n, 256)
    return key

In [41]:
n, e, d = generate_keys()

In [42]:
key = generate_sym_key(n)
print("Key:", key)
c_key = encode_sym_key(key, e, n)
print("\nEncoded key:", c_key)

Key: b'w\xc26\xd4\xca\x99\x04wX\xfb\xa0\xc3I\xc4\xd4\xef\xba;\xbb\xb517B1\x03\xd7\xa0\xb2H\xde(\x05'

Encoded key: 9391135878242844221224416004999813130436309188556744359240069983758952798749930115856453870202923067641042264540691410800708512255038563104623814585794920924175158677448598116292092945333665675174497359288386850984124213676983136286362639752069692480407214900484832148645429300941759033453822235105938115618591660532473755743038836713969069550774268326310376853436859394201703169648963211682365429867850199796939655554278069118098579213383934357970454832460058999526877968187894821258987124478771808616718009091081105253239495532334343505268652270751566475978975867137315128044679211819856849125553198223209808272177


In [43]:
d_key = decode_sym_key(c_key, d, n)
print("Decoded key:", d_key)

Decoded key: b'w\xc26\xd4\xca\x99\x04wX\xfb\xa0\xc3I\xc4\xd4\xef\xba;\xbb\xb517B1\x03\xd7\xa0\xb2H\xde(\x05'
